In [ ]:
from ultralytics import YOLO
from utilities.text_extraction.detect_roi import *
model = YOLO(MODEL_PATH)

In [ ]:
img_path = r"C:\Users\Levi\Desktop\clock-roi-dataset-small\generalized_test_images\nba_1.png"
model(img_path, save = True)

# Image Preprocessing
***

In [ ]:
# PREPROCESS AN IMAGE

import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import copy


def preprocess_image_test(image):
    """Preprocess a ROI for OCR."""

    def change_dpi(image, target_dpi=95):
        """95 is the magic number, font height should be 30-33 px for best results."""
        
        try:
            image = Image.fromarray(image)
            current_dpi = image.info.get("dpi", (72, 72))
            scale_factor = target_dpi / current_dpi[0]
            new_width = int(image.width * scale_factor)
            new_height = int(image.height * scale_factor)
            resized_image = image.resize((new_width, new_height))
            resized_image.info["dpi"] = (target_dpi, target_dpi)
            return np.array(resized_image)
        except Exception as e:
            raise Exception("An error while preprocessing a frame:", str(e))

    scaled_image = change_dpi(image)
    gray = cv2.cvtColor(scaled_image, cv2.COLOR_BGR2GRAY)
    thresh = cv2.threshold(gray, 135, 255, cv2.THRESH_BINARY)[1]
    kernel = np.ones((3, 3), np.uint8)
    result = cv2.dilate(thresh, kernel, iterations=1)

    result_c1 = copy.copy(result)
    result_c2 = copy.copy(result)

    black_pixels = result_c1[np.where(result_c1 == 0)].size
    white_pixels = result_c2[np.where(result_c2 == 255)].size

    if black_pixels > white_pixels:
        result = cv2.bitwise_not(result)

    cv2.imwrite(r"demos\example_images\preprocessed_images\pp.png", result)
    return result


In [ ]:
from PIL import Image

outpath = r"demos\example_images\preprocessed_images\pp.png"
img_path = r"demos\example_images\example_rois\example_roi_1.PNG"

image = Image.open(img_path)
image = np.array(image)
result = preprocess_image_test(image)

# cv2.imwrite(outpath, result)


# Text Extraction
***

In [3]:
from utilities.text_extraction.timestamp_extraction import extract_timestamps_from_frame
from PIL import Image
import numpy as np

frame = Image.open(r"demos\example_images\example_rois\example_roi_1.PNG")
arr = np.array(frame)

results = extract_timestamps_from_frame(arr)
print(results)

wonocurg RUS 75 62 NED 4th 241

GAMEDAYS =X ericaWc

-



# ROI Detection
***

In [ ]:
from video import Video
from utilities.text_extraction.detect_roi import detect_roi

video_path = r"demos\example_videos_DNC\clock_roi_eur_4.mp4"
video = Video(video_path)

detect_roi(video)
